In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


'C:\Users\Lokesh' is not recognized as an internal or external command,
operable program or batch file.


In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

delta = "https://www.consumeraffairs.com/travel/delta.htm?page="
air_canada = "https://www.consumeraffairs.com/travel/air_canada.html?page="
air_india = "https://www.consumeraffairs.com/travel/air_india.html?page="
KLM = "https://www.consumeraffairs.com/travel/klm.html?page="
emirates = "https://www.consumeraffairs.com/travel/emirates.html?page="
american = "https://www.consumeraffairs.com/travel/american_airlines.htm?page="
qatar = "https://www.consumeraffairs.com/travel/qatar.html?page="
alaska = "https://www.consumeraffairs.com/travel/alaska.html?page="

brands = [delta,air_canada,air_india,KLM,emirates,american,qatar,alaska]
names = ["Delta","Air Canada","Air India","KLM","Emirates","American Airlines","Qatar Airways","Alaska Airlines"]
col = ['Air Line','Rating','Review Date','Review']
data_df = pd.DataFrame(columns = col)
flag = 0

for name in brands:
    for i in range(10):
        link = name+str(i+1)
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'lxml')
        for item in soup.find_all('div', class_='rvw js-rvw'):
            brand = names[flag]
            try:
                star = str(item.find('div',class_='rvw__hdr-stat')).split('Rated with ')[1].split(' star')[0]
                r_date = getattr(item.find('span',class_='ca-txt-cpt'),'text',None).split('review:')[1]
                review = getattr(item.find('div',class_='rvw-bd'),'text',None).split(r_date)[1].split('Read full review')[0]
            except IndexError:
                pass
            
            data_df = data_df.append({
                "Air Line" : brand,
                "Rating" : star,
                "Review Date" : r_date,
                "Review" : review
            }, ignore_index=True)
            
    flag = flag+1

In [29]:
data_df['Air Line'].value_counts()

Alaska Airlines      285
Delta                285
American Airlines    280
Air Canada           270
Air India            240
Emirates             240
Qatar Airways        240
KLM                  226
Name: Air Line, dtype: int64

In [30]:
data_df

,Air Line,Rating,Review Date,Review
0,Delta,5,"March 30, 2021","Simply put, I absolutely love Delta Airlines. ..."
1,Delta,4,"March 23, 2021","Overall, I love Delta for the culture of Safet..."
2,Delta,5,"Dec. 21, 2020",I had a great experience with delta on Friday ...
3,Delta,5,"Nov. 27, 2020","Delta airlines exceeded any expectation, they ..."
4,Delta,5,"Sept. 30, 2020","In February 2020, I flew Delta knowing I had a..."
...,...,...,...,...
2061,Alaska Airlines,5,"July 31, 2017","I just got back from a 2 week vacation, and I ..."
2062,Alaska Airlines,1,"July 31, 2017",I fly from Missoula to Fairbanks on Alaska Air...
2063,Alaska Airlines,2,"July 17, 2017",I tried to voice my concerns via the telephone...
2064,Alaska Airlines,1,"July 12, 2017",Alaska airlines is horrible. Due to custody my...


In [31]:
import feedparser
import re
import pandas as pd
import nltk.data
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from nltk import word_tokenize
from nltk.corpus import stopwords 
import string
from pymongo import MongoClient

In [32]:
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Lokesh
[nltk_data]     Grandhe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Lokesh
[nltk_data]     Grandhe\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Lokesh
[nltk_data]     Grandhe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
data_df['clean_review'] = data_df['Review'].str.replace('[^\w\s]','') ##Removing punctuations
data_df['clean_review'] = data_df['clean_review'].str.replace('\d+', '') ##Removing numbers
data_df['clean_review'] = data_df['clean_review'].apply(lambda x: " ".join(x.lower() for x in x.split())) ##Lower casing

stop = stopwords.words('english')
data_df['Cleaned'] = data_df['clean_review'].apply(lambda Y: " ".join(Y for Y in Y.split() if Y not in stop)) ##Removing stop words
data_df.drop(['clean_review'],axis=1)

,Air Line,Rating,Review Date,Review,Cleaned
0,Delta,5,"March 30, 2021","Simply put, I absolutely love Delta Airlines. ...",simply put absolutely love delta airlines make...
1,Delta,4,"March 23, 2021","Overall, I love Delta for the culture of Safet...",overall love delta culture safety service comp...
2,Delta,5,"Dec. 21, 2020",I had a great experience with delta on Friday ...,great experience delta friday th year old son ...
3,Delta,5,"Nov. 27, 2020","Delta airlines exceeded any expectation, they ...",delta airlines exceeded expectation helped tra...
4,Delta,5,"Sept. 30, 2020","In February 2020, I flew Delta knowing I had a...",february flew delta knowing medical bag bring ...
...,...,...,...,...,...
2061,Alaska Airlines,5,"July 31, 2017","I just got back from a 2 week vacation, and I ...",got back week vacation want thank alaska airli...
2062,Alaska Airlines,1,"July 31, 2017",I fly from Missoula to Fairbanks on Alaska Air...,fly missoula fairbanks alaska airlines company...
2063,Alaska Airlines,2,"July 17, 2017",I tried to voice my concerns via the telephone...,tried voice concerns via telephone avail seate...
2064,Alaska Airlines,1,"July 12, 2017",Alaska airlines is horrible. Due to custody my...,alaska airlines horrible due custody son flyin...


In [34]:
# Sentiment score from TextBlob
def textblob_analysis(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

# Sentiment score from Vader
def vader_analysis(txt):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(txt)
    return vs['compound']

data_df['Sentiment'] = data_df['Cleaned'].apply(lambda x : (vader_analysis(x) + textblob_analysis(x))/2)

data_df['Sentiment Status'] = data_df['Sentiment'].apply(lambda X : 'Positive' if X > 0.1 else 'Negative' if X < -0.1 else 'Neutral')

In [35]:
data_df

,Air Line,Rating,Review Date,Review,clean_review,Cleaned,Sentiment,Sentiment Status
0,Delta,5,"March 30, 2021","Simply put, I absolutely love Delta Airlines. ...",simply put i absolutely love delta airlines wh...,simply put absolutely love delta airlines make...,0.573114,Positive
1,Delta,4,"March 23, 2021","Overall, I love Delta for the culture of Safet...",overall i love delta for the culture of safety...,overall love delta culture safety service comp...,0.674212,Positive
2,Delta,5,"Dec. 21, 2020",I had a great experience with delta on Friday ...,i had a great experience with delta on friday ...,great experience delta friday th year old son ...,0.513519,Positive
3,Delta,5,"Nov. 27, 2020","Delta airlines exceeded any expectation, they ...",delta airlines exceeded any expectation they h...,delta airlines exceeded expectation helped tra...,0.619165,Positive
4,Delta,5,"Sept. 30, 2020","In February 2020, I flew Delta knowing I had a...",in february i flew delta knowing i had a medic...,february flew delta knowing medical bag bring ...,0.479013,Positive
...,...,...,...,...,...,...,...,...
2061,Alaska Airlines,5,"July 31, 2017","I just got back from a 2 week vacation, and I ...",i just got back from a week vacation and i wan...,got back week vacation want thank alaska airli...,0.739583,Positive
2062,Alaska Airlines,1,"July 31, 2017",I fly from Missoula to Fairbanks on Alaska Air...,i fly from missoula to fairbanks on alaska air...,fly missoula fairbanks alaska airlines company...,0.556003,Positive
2063,Alaska Airlines,2,"July 17, 2017",I tried to voice my concerns via the telephone...,i tried to voice my concerns via the telephone...,tried voice concerns via telephone avail seate...,0.493367,Positive
2064,Alaska Airlines,1,"July 12, 2017",Alaska airlines is horrible. Due to custody my...,alaska airlines is horrible due to custody my ...,alaska airlines horrible due custody son flyin...,-0.326379,Negative


In [36]:
data_df['Sentiment Status'].value_counts()

Negative    1020
Positive     879
Neutral      167
Name: Sentiment Status, dtype: int64

In [37]:
data_df.to_excel('airlines.xlsx')